In [2]:
using VCFTool

In [3]:
input_dir = "../input/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_738_1000_file_path = joinpath(input_dir, "738_variants_1000.vcf.gz")

vcf_file_path_to_use = vcf_738_1000_file_path

"../input/738_variants_1000.vcf.gz"

## Search by Variant

In [4]:
vcf = make_vcf_ndsparse(vcf_file_path_to_use)

2-d NDSparse with 1000 values (8 field named tuples):
    Dimensions#  colname  type
──────────────────
1  CHROM    String
2  POS      String
    Values#   colname  type
───────────────────
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     String
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  GERM     String

In [11]:
# https://www.snpedia.com/index.php/Rs1042522

using Dates

start_time = now()

for value in vcf["chr1", :]
    
    genotype = []
    
    allele_ref = value[:REF]
    
    allele_alt = value[:ALT]
    
    numeric_genotype = split(split(value[:GERM], ":")[1], "/")
    
    println(numeric_genotype)
    
    println(allele_ref)
    
    println(allele_alt)

    counter = 1
    
        while counter < 3
        
            println(counter)
        
            if "0" == numeric_genotype[counter]

                push!(genotype, allele_ref)

            elseif "1" == numeric_genotype[counter]

                push!(genotype, split(allele_alt, ",")[1])

            elseif "2" == numeric_genotype[counter]

                push!(genotype, split(allele_alt, ",")[2])
        
            end    
    
        counter += 1
        
        end

    println(genotype)
    
    println("\n")
    
end
  
end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")

SubString{String}["0", "1"]
C
T
1
2
Any["C", "T"]


SubString{String}["0", "1"]
T
A
1
2
Any["T", "A"]


SubString{String}["0", "1"]
T
G
1
2
Any["T", "G"]


SubString{String}["0", "1"]
C
T
1
2
Any["C", "T"]


SubString{String}["0", "1"]
A
G
1
2
Any["A", "G"]


SubString{String}["1", "2"]
A
AACAC,AACACAC
1
2
Any["AACAC", "AACACAC"]


SubString{String}["0", "1"]
AC
A
1
2
Any["AC", "A"]


SubString{String}["1", "1"]
C
T
1
2
Any["T", "T"]


SubString{String}["0", "1"]
G
C
1
2
Any["G", "C"]


SubString{String}["0", "1"]
A
G
1
2
Any["A", "G"]


SubString{String}["0", "1"]
C
G
1
2
Any["C", "G"]


SubString{String}["0", "1"]
T
C
1
2
Any["T", "C"]


SubString{String}["0", "1"]
C
CA
1
2
Any["C", "CA"]


SubString{String}["0", "1"]
C
T
1
2
Any["C", "T"]


SubString{String}["0", "1"]
C
CGTGT
1
2
Any["C", "CGTGT"]


SubString{String}["0", "1"]
G
A
1
2
Any["G", "A"]


SubString{String}["0", "1"]
T
C
1
2
Any["T", "C"]


SubString{String}["0", "1"]
T
C
1
2
Any["T", "C"]


SubString{String}["0", "1"]
C


LoadError: BoundsError: attempt to access 1-element Array{SubString{String},1} at index [2]

In [ ]:
import Pkg; Pkg.add("Dagger")

using Dagger

Dagger.save(vcf, "../input/test_vcf_ndsparse_save.vcf")

In [ ]:
using Dagger

dagger_load_vcf = Dagger.load("../input/test_vcf_ndsparse_save.vcf")




## Search by Gene

## Search by Region

## Tabix Regions

In [ ]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)

## JuliaDB

An unzipped VCF file can be loaded into an indextable or ndsparse array directly using loadtable() or ndsparese() respectively. However, its much faster to query variants if they are in an ndsparse array because the chromosome and position are both part of the index.

In [ ]:
vcf_ndsparse_738 = make_ndsparse("../input/738_variants.vcf.gz")

In [ ]:
for item in vcf_ndsparse_738
    
    println(item) 

    println(item[:REF])

    end
    
end